In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# set the modified tf session as backend in keras
import keras
keras.backend.tensorflow_backend.set_session(get_session())


from model import *
from data import *
from keras import models
from keras import layers
from keras.layers import Input, Dense, Flatten, Reshape, Permute
from keras.models import Model
from keras.layers import Activation
from keras.layers import Lambda
import keras.backend as K
import keras
import PIL
import scipy

print ('keras.__version__='+str(keras.__version__))
print ('tf.__version__='+str(tf.__version__))
print ('PIL.__version__='+str(PIL.__version__))
print ('np.__version__='+str(np.__version__))
print ('scipy.__version__='+str(scipy.__version__))
print ('Using GPU '+str(os.environ["CUDA_VISIBLE_DEVICES"])+'  Good luck...')


Using TensorFlow backend.


keras.__version__=2.2.4
tf.__version__=1.12.0
PIL.__version__=5.4.1
np.__version__=1.16.2
scipy.__version__=1.2.1
Using GPU 2  Good luck...


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

myGene = trainGenerator(8,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)

model = unet()

model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
reduceLROnPlateau = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, verbose=0, 
                                                      mode='auto', min_delta=0.0001, cooldown=0, min_lr=0) 
earlyStopping = keras.callbacks.EarlyStopping(monitor='loss',min_delta=0.005,patience=5,verbose=0, mode='auto') 

#fit_generator(generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)

model.fit_generator(generator=myGene,steps_per_epoch=2000,epochs=25,
                    #validation_data=myGeneTest, validation_steps=2000,
                    callbacks=[model_checkpoint,reduceLROnPlateau,earlyStopping])

Epoch 1/25
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
2000/2000 [==============================] - 368s 184ms/step - loss: 0.1852 - acc: 0.9185

Epoch 00001: loss improved from inf to 0.18520, saving model to unet_membrane.hdf5
Epoch 2/25
2000/2000 [==============================] - 358s 179ms/step - loss: 0.1020 - acc: 0.9555

Epoch 00002: loss improved from 0.18520 to 0.10203, saving model to unet_membrane.hdf5
Epoch 3/25
1182/2000 [================>.............] - ETA: 2:26 - loss: 0.0826 - acc: 0.9641

### Train with npy file

In [ ]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [ ]:
testGene = testGenerator("data/membrane/test")
model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene,30,verbose=1)
saveResult("data/membrane/test",results)